# Practical Session 2
### Kernel Methods for Machine Learning

Written by Yunlong Jiao / Romain Menegaux, 19 May 2020

In [1]:
# setup
import numpy as np

In [2]:
import sys
print(sys.version)

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [3]:
import sklearn
from sklearn import linear_model as lm
sklearn.__version__

'0.22.1'

## Tasks

1. Implement (naive) solvers to Ridge Regression, Weighted Ridge Regression and Logistic Ridge Regression (using Iteratively Reweighted Least Squares). See notes for the mathematical derivation.
2. Simulate some toy data to check if our solvers give correct solutions.

In [4]:
# Toy data
np.random.seed(42)
n = 100
p = 10
X = np.random.normal(0, 1, (n, p))
X = sklearn.preprocessing.scale(X)
beta_star = np.random.normal(0, 1, p)
y = X.dot(beta_star) + 0.2 * np.random.normal(0, 1, n)

def compare(beta1, beta2):
    print('''
Our solver:
{}
Scikit-learn:
{}

Difference between the two:
{}
        '''.format(beta1, beta2, np.sum((beta1-beta2)**2))
    )

In [ ]:
# def solveRR(y, X, lam):
# n, p = X.shape
# assert (len(y) == n)

# A = X.T.dot(X)
# # Adjust diagonal due to Ridge
# A[np.diag_indices_from(A)] += lam * n
# b = X.T.dot(y)

# # Hint:
# beta = np.linalg.solve(A, b)
# # Finds solution to the linear system Ax = b
# return (beta) 
# # Weighted Ridge Regression (WRR)
# def solveWRR(y, X, w, lam):
# n, p = X.shape
# assert (len(y) == len(w) == n)

# y1 = np.sqrt(w) * y
# X1 = (np.sqrt(w) * X.T).T
# # Hint:
# # Find y1 and X1 such that:
# beta = solveRR(y1, X1, lam)
# return (beta) 


# Logistic Ridge Regression (LRR)
# def solveLRR(y, X, lam):
# n, p = X.shape
# assert (len(y) == n)

# # Parameters
# max_iter = 100
# eps = 1e-3
# sigmoid = lambda a: 1/(1 + np.exp(-a))

# # Initialize
# beta = np.zeros(p)

# # Hint: Use IRLS
# for i in range(max_iter):
# beta_old = beta
# f = X.dot(beta_old)
# w = sigmoid(f) * sigmoid(-f)
# z = f + y / sigmoid(y*f)
# beta = solveWRR(z, X, w, 2*lam)
# # Break condition (achieved convergence)
# if np.sum((beta-beta_old)**2) < eps:
# break
# return (beta) 


## Solutions

**Ridge Regression (RR)**

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^n$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \|y - X \beta\|^2 + \lambda \|\beta\|^2 \,.
$$

In [5]:
# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    A = X.T.dot(X)
    #adjust diagonal due to ridge
    A[np.diag_indices_from(A)] += lam*n
    b =X.T.dot(y)
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    
    return (beta)

**Try it out:**

In [6]:
lam = 0.1

# Our solver
beta1 = solveRR(y, X, lam)

# Python solver
alpha = lam * X.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X, y).coef_

# Check
compare(beta1, beta2)


Our solver:
[ 1.27929172  0.78935356  0.05064497 -0.55474398  0.65276533  0.32637554
  0.765293    0.63326617  0.97285396 -0.5294559 ]
Scikit-learn:
[ 1.27929172  0.78935356  0.05064497 -0.55474398  0.65276533  0.32637554
  0.765293    0.63326617  0.97285396 -0.5294559 ]

Difference between the two:
1.9841893252049497e-31
        


**Weighted Ridge Regression (WRR)**

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^n$, and weights $w \in \mathbb{R}^n_+$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n w_i (y_i - \beta^\top x_i)^2 + \lambda \|\beta\|^2 \,.
$$
Think of $w_i$ as important or confidence you have in poit $i$.


**Goal:** Express the objective as a regular Ridge Regression (RR)

In [7]:
# Weighted Ridge Regression (WRR)
def solveWRR(y, X, w, lam):
    n, p = X.shape
    assert (len(y) == len(w) == n)
#     w = np.diag(w)
    y1 = np.sqrt(w)*y
    X1 = (np.sqrt(w)* X.T).T
    

    # Hint:
    # Find y1 and X1 such that:
    beta = solveRR(y1, X1, lam)
    return (beta)

**Try it out:**

In [8]:
lam = 0.1
w = np.random.rand(len(y))

# Our solver
beta1 = solveWRR(y, X, w, lam)

# Python solver
alpha = lam * X.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X, y, sample_weight=w).coef_

# Check
compare(beta1, beta2)


Our solver:
[ 1.21907902  0.69464471  0.05119819 -0.50155617  0.60130538  0.2718474
  0.6571524   0.60033163  0.862139   -0.50913719]
Scikit-learn:
[ 1.21907902  0.69464471  0.05119819 -0.50155617  0.60130538  0.2718474
  0.6571524   0.60033163  0.862139   -0.50913719]

Difference between the two:
3.081969393505674e-31
        


**Logistic Ridge Regression (LRR)**

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \{-1,+1\}^n$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n \log (1+e^{-y_i \beta^\top x_i}) + \lambda \|\beta\|^2 \,.
$$

Let $\sigma(x) = \frac{1}{1 + e^{-x}}$ be the sigmoid function.

Compute $\sigma'(x) = \frac{e^{-x}}{(1+ e^{-x})^2}=\sigma(x) (1-\sigma(x) $

Rewriting $J$:
$$
J(\beta) = - \frac{1}{n} \sum_{i=1}^n {\log(\sigma(y_i\beta^\top x_i))} + 2\lambda \beta \,.
$$

Compute its gradient $\nabla J$, and its Hessian $\nabla^2 J$
$$$$
**Computing the gradien**:

$\nabla J (\beta) = \frac{- 1}{n}\sum(y_i) \sigma(-y_i\beta^T x_i)x_i + 2 \lambda \beta $

**Hessian matrix**:

$\nabla ^2 J(\beta)= \nabla (\nabla J(\beta))$
$$$$
$ \nabla ^2 J(\beta) =\frac{-1}{n}X^T WX + 2 \lambda I$

#### Solving for optimal $\beta$ using Newton-Raphson
$$
\beta^{new} \leftarrow \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})
$$ 
This agoritm converge to the optimal $\beta$

$\approx$ gradient descent with adapted step size.


Show that each step is equivalent to solving a weighted ridge regression (WRR). Hence we can compute $\beta$ without the Hessian.


<font color='green'>Quadratic approximation to $J$</font>:

$$
J(\beta) \approx J_q(\beta) = J(\beta^{old}) + (\beta - \beta^{old})^\top \nabla J(\beta^{old}) + \frac{1}{2} (\beta - \beta^{old})^\top \nabla^2 J(\beta^{old}) (\beta - \beta^{old})
$$

**lemma**: $\min_\beta J_q(\beta) = \beta^{new}$

*proof*: we know that $\beta ^{new} = \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})$

So $\nabla J_q (\beta) = \nabla J(\beta^{old}) + \nabla^{2}J(\beta^{old})(\beta - \beta^{old})   $

$\nabla J_q (\beta)=0 \iff \nabla J(\beta^{old}) + \nabla^{2}J(\beta^{old})(\beta - \beta^{old}) =0 $
$\iff \beta = \beta^{old} - (\nabla^{2}J(\beta^{old}))^{-1} \nabla J (\beta^{old})$

Next **show that $J_q$ is a WRR objective**
Let us write   $J_q$ 

In [ ]:
 sigmoid =lambda a: 1/(1+np.exp(-a))
sigmoid(2)

In [9]:
# Logistic Ridge Regression (LRR)
def solveLRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    #beta_old = 0.001
    
    #parameters
    max_iter = 100
    eps = 1e-3
    sigmoid = lambda a: 1/(1+np.exp(-a))
    
    #initialization
    beta= np.zeros(p)
       
            
    # Hint: Use IRLS
    for i in range(max_iter):
    #     ...
        beta_old = beta 
        f = X.dot(beta_old)
        w = (sigmoid(f)*sigmoid(-f))
        z = f + y/sigmoid(y*f)
#         print(X.shape, z.shape, w.shape)
        beta = solveWRR(z, X, w, 2*lam)
    #break condition
        if np.sum((beta - beta_old)**2) <eps:
            break

#     if (some condition)
    return (beta)

**Try it out:**

In [10]:
y_bin = np.sign(y) # Binarize targets
lam = 0.1

# Our solver
beta1 = solveLRR(y_bin, X, lam)

# Python solver
alpha = 2 * lam * X.shape[0]
model = lm.LogisticRegression(C=1/alpha, fit_intercept=False)
beta2 = model.fit(X, y_bin).coef_

# Check
compare(beta1, beta2)


Our solver:
[ 0.49767127  0.31536865 -0.08695284 -0.16928256  0.27290418  0.15201934
  0.35505561  0.31972318  0.30170835 -0.39445233]
Scikit-learn:
[[ 0.49767184  0.31536829 -0.08695313 -0.16928249  0.2729044   0.15201826
   0.3550554   0.31972255  0.30170796 -0.39445232]]

Difference between the two:
2.3618668897770345e-12
        


### Mini Data Challenge

We will try to predict whether patients have breast cancer.

We use scikit-learn's [breast cancer dataset](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset)

30 features, 569 samples, 2 labels ('malignant' or 'benign')

In [11]:
# Load data and split into training / validation sets
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True)
X = sklearn.preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
X.shape

(569, 30)

In [19]:
X.mean(axis=0)

array([-3.16286735e-15, -6.53060890e-15, -7.07889127e-16, -8.79983452e-16,
        6.13217737e-15, -1.12036918e-15, -4.42138027e-16,  9.73249991e-16,
       -1.97167024e-15, -1.45363120e-15, -9.07641468e-16, -8.85349205e-16,
        1.77367396e-15, -8.29155139e-16, -7.54180940e-16, -3.92187747e-16,
        7.91789988e-16, -2.73946068e-16, -3.10823423e-16, -3.36676596e-16,
       -2.33322442e-15,  1.76367415e-15, -1.19802625e-15,  5.04966114e-16,
       -5.21317026e-15, -2.17478837e-15,  6.85645643e-16, -1.41265636e-16,
       -2.28956670e-15,  2.57517109e-15])

In [12]:
# Fit our model and compute its parameters
lam = 0.01
beta = solveLRR(y_train, X_train, lam)

In [20]:
# Compute predicted probabilities and classes
sigmoid  = lambda a: 1/(1+np.exp(-a))
pred = beta.T.dot(X_test.T)
probas_pred = sigmoid(pred)
y_pred = np.round(probas_pred)

In [21]:
from sklearn.metrics import accuracy_score, roc_auc_score

print("Our model's performance:")
print('Accuracy: {:.2%}'.format(accuracy_score(y_test, y_pred)))
print('AUC: {:.2%}'.format(roc_auc_score(y_test, probas_pred)))

Our model's performance:
Accuracy: 97.87%
AUC: 99.75%


In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97        67
           1       0.99      0.98      0.98       121

    accuracy                           0.98       188
   macro avg       0.97      0.98      0.98       188
weighted avg       0.98      0.98      0.98       188



In [34]:
# pip install bqplot

In [32]:
import bqplot.pyplot as plt
# from bqplot import pyplot as plt

In [35]:

plt.figure(title = 'feature importances')
plt.bar(np.arange(len(beta)), beta)
plt.show()

In [36]:
plt.figure(1)
n = 200
x = np.linspace(0.0, 10.0, n)
y = np.cumsum(np.random.randn(n))
plt.plot(x,y, axes_options={'y': {'grid_lines': 'dashed'}})
plt.show()